In [1]:
# Feature engineering and model selection
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
# Supress all warning
import warnings
import joblib
warnings.filterwarnings('ignore')

# Analytical helper functions to declutter the notebook
from analysis.ml.custom_transformers import *
from analysis.ml.data import get_ml_features, get_ml_dataset

In [2]:
# Construct base datasets for model training
# get_ml_dataset(resample_period='1d')

In [3]:
# Pipeline for feature engineering and modeling
feature_engineering_pipeline = Pipeline([
    ('time_features', TimeFeatures()),

    ('returns_features', ReturnsFeatures(
        window_sizes = [1, 7],
        lookback_windows = [30]
    )),

    ('risk_features', RiskFeatures(
        windows = [1, 7],
        lookback_windows = [30]
    )),

    ('trade_features', TradeFeatures(
        windows = [1, 7],
        lookback_windows = [30]
    )),

    ('spot_futures_features', SpotFuturesInteractionFeatures(
        windows = [1, 7],
        lookback_windows = [30]
    )),

    ('rolling_z_score', RollingZScoreScaler(window_sizes = [30]))
])

Calculating returns for 1000SATS_USDT_BINANCE with window size 1...
Calculating returns for 1000SATS_USDT_BINANCE with window size 7...
Calculating returns for 1INCH_USDT_BINANCE with window size 1...
Calculating returns for 1INCH_USDT_BINANCE with window size 7...
Calculating returns for 1MBABYDOGE_USDT_BINANCE with window size 1...
Calculating returns for 1MBABYDOGE_USDT_BINANCE with window size 7...
Calculating returns for AAVE_USDT_BINANCE with window size 1...
Calculating returns for AAVE_USDT_BINANCE with window size 7...
Calculating returns for ACA_USDT_BINANCE with window size 1...
Calculating returns for ACA_USDT_BINANCE with window size 7...
Calculating returns for ACE_USDT_BINANCE with window size 1...
Calculating returns for ACE_USDT_BINANCE with window size 7...
Calculating returns for ACH_USDT_BINANCE with window size 1...
Calculating returns for ACH_USDT_BINANCE with window size 7...
Calculating returns for ACM_USDT_BINANCE with window size 1...
Calculating returns for A

In [4]:
get_ml_features(feature_engineering_pipeline = feature_engineering_pipeline)    

Processing symbol_id: 1000SATS_USDT_BINANCE (1/368)
Processing symbol_id: 1INCH_USDT_BINANCE (2/368)
Column set difference: set()

Processing symbol_id: 1MBABYDOGE_USDT_BINANCE (3/368)
Column set difference: set()

Processing symbol_id: AAVE_USDT_BINANCE (4/368)
Column set difference: set()

Processing symbol_id: ACA_USDT_BINANCE (5/368)
Column set difference: set()

Processing symbol_id: ACE_USDT_BINANCE (6/368)
Column set difference: set()

Processing symbol_id: ACH_USDT_BINANCE (7/368)
Column set difference: set()

Processing symbol_id: ACM_USDT_BINANCE (8/368)
Column set difference: set()

Processing symbol_id: ADA_USDT_BINANCE (9/368)
Column set difference: set()

Processing symbol_id: ADX_USDT_BINANCE (10/368)
Column set difference: set()

Processing symbol_id: AERGO_USDT_BINANCE (11/368)
Column set difference: set()

Processing symbol_id: AEUR_USDT_BINANCE (12/368)
Column set difference: set()

Processing symbol_id: AEVO_USDT_BINANCE (13/368)
Column set difference: set()

Proces